# 自定函式

R 和其他語言一樣，可以自行定義函式(function)，在基本操作的章節裡頭我們曾提過[使用 R 內建的函式](https://hackmd.io/s/S1CLyIynW#使用函式及表示式)。當你需要重複處理一些程序時，自己撰寫函式就可以反覆使用程式碼，不需要每次都得執行落落長的 R 程式。舉例來說，當計算 [Shannon-Wiener 多樣性指數](https://zh.wikipedia.org/wiki/多樣性指數)時，我們用 R 來計算會怎麼做呢？

Shannon-Wiener 多樣性指數的公式如下：

$$
H' = - \sum_{i=1}^{S} P_i \cdot ln (P_i)
$$

其中 S 是一地的總物種數， $P_i$ 則是第 $i$ 個物種之數量佔總物種數量的比例

若臺灣某森林樣區裡頭木本植物的數量如下表所示，其中第一個欄位是物種名(species)，第二個欄位則是該物種在樣區裡頭的數量(individuals)：

| species    | individuals   |
|------------|---------|
| 樹杞       | 17      |
| 糊樗       | 1       |
| 黃楊       | 2       |
| 鼠刺       | 6       |
| 九節木     | 22      |
| 大明橘     | 9       |
| 山紅柿     | 2       |
| 山豬肝     | 1       |
| 水金京     | 4       |
| 烏心石     | 3       |
| 燈稱花     | 4       |
| 牛奶榕     | 2       |
| 豬母乳     | 1       |
| 豬腳楠     | 15      |
| 鵝掌柴     | 12      |
| 台灣楊桐   | 2       |
| 台灣樹參   | 4       |
| 台灣糊樗   | 1       |
| 呂宋莢蒾   | 1       |
| 小花鼠刺   | 1       |
| 小葉樹杞   | 1       |
| 廣東油桐   | 1       |
| 金毛杜鵑   | 3       |
| 倒卵葉冬青 | 1       |
| 圓葉雞屎樹 | 2       |
| 奧氏虎皮楠 | 2       |
| 小梗木薑子 | 1       |
| 森氏紅淡比 | 1       |
| 琉球雞屎樹 | 2       |
| 細葉饅頭果 | 1       |

原始的資料可至[github 生態資訊學上課資料](https://github.com/mutolisp/ecoinformatics_course/blob/master/data/twforest.csv)中下載。

所以若要計算這個樣區的 Shannon-Wiener 多樣性指數，R 的 code 如下：

In [1]:
# 載入 data.table
library(data.table)

twforest <- fread('https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/data/twforest.csv')

# 計算總物種數
S <- twforest[, sum(individuals) ]
# 計算個別物種數佔總物種數之比例
p_i <- twforest[, individuals/S ]
# 最後加總並算出 Shannon-Wiener (H) 多樣性指數
H <- -sum(p_i * log(p_i))
H

[1] 2.827801

[本範例 code 下載](https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/demos/custom_functions.R)

可是每次有一個新的資料進來，計算就得要重新執行一次 (也就是執行上面的程式碼)，此時就可以把上面的步驟包裝成一個函式，簡化工作。試著想想：如果有個函式叫做 ```shannon()```，每當我們想要計算 Shannon-Wiener 多樣性指數的時候，就呼叫它計算，這樣是不是很方便呢？接下來我們將針對函式的結構及撰寫來做介紹。

## 函式結構

一般 R 函式的結構如下：

```{r}=
myFunction <- function(參數1, 參數2, ...){
    陳述(statements)
    return(objects)
}
```

```function(){}``` 括號內的是參數(parameters)，大括號則是你想執行的程式碼(statement)，如果你希望這個函式計算完之後能回傳(return)一些東西(數值或物件等)，也可以在函式內的最後一行加上```return(你要傳回的物件)```。

**範例一、Hello World**

In [3]:
### 生態資訊學範例

helloWorld <- function(){
    print('Hello World!')
}

helloWorld()

[1] "Hello World!"


[本範例 code 下載](https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/demos/func_HelloWorld.R)

在這個範例中，自訂的函式沒有參數，內容只有印出「Hello World!」

**範例二、加入參數的函式**

我們稍微改一下上面 Hello World 的範例，加上參數

In [5]:
helloWorld <- function(username){
  sprintf('Hello! %s', username)
}

helloWorld('小熊維尼')

[1] "Hello! 小熊維尼"

[本範例 code 下載](https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/demos/func_HelloWorld2.R)

這個例子中，我們用到 ```sprintf``` 這個 C 類型的字串格式化函式，裡頭的 `%s` 代表變數(s 代表 string)，變數的內容是 ```username```。所以當使用者輸入 ```helloWorld('小熊維尼')``` 時，這個函式就會印出 "Hello! 小熊維尼"。

**範例三、設定參數的預設值**

你可能會想問，參數是否可以有預設值呢？答案是可以的！只要在參數後方加上```=value```，這個 ```value``` 就是該參數的預設值，舉例來說：

In [6]:
helloWorld <- function(username='小熊維尼'){
  sprintf('Hello! %s', username)
}

helloWorld()

[1] "Hello! 小熊維尼"

[本範例 code 下載](https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/demos/func_HelloWorld3.R)

若這個函式內有預設值的話，即使呼叫函數沒有加上參數也能夠執行，所以上面的執行結果也會印出 "Hello! 小熊維尼"。

**範例四、Shannon-Wiener 多樣性指數計算**

回到本節最前面的實例，如果我們要把 Shannon-Wiener 多樣性指數包成一個函式該怎麼做呢？以本例來說流程可以切為三個步驟：

1. 分析這個函式需要輸入什麼參數？
2. 內容該如何計算？
3. 需要輸出哪些東西？

在第一個步驟中，我們需要有物種和其個體數的資料，資料格式共有兩個欄位。接下來第二步驟就是先算出總物種個體數，以及個別物種的個體數佔總物種數的比例。最後則是把計算出來的數值輸出，所以函式可以這樣寫：

In [7]:
ShannonWiener <- function(filePath) {
  twforest <- fread(filePath)
  # 計算總物種數
  S <- twforest[, sum(individuals) ]
  # 計算個別物種數佔總物種數之比例
  p_i <- twforest[, individuals/S ]
  # 最後加總並算出 Shannon-Wiener (H') 多樣性指數
  H <- -sum(p_i * log(p_i))
  return(H)
}

[本範例 code 下載](https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/demos/func_ShannonWiener.R)

在第一行內，```filePath```是我們要輸入的參數，也就是輸入的檔案路徑。所以在第二行就使用 ```fread()``` 來讀取這個參數。之後的4–8行和我們之前的程式碼相同。第九行則是傳回 ```H``` 這個變數，也就是計算完的 Shannon-Wiener 多樣性指數。

## 分享你的函式

當撰寫完你的函式之後，如果你想和別人分享，可以把它包成套件(package)上傳 CRAN 或是 github 和其他人分享。但包套件是比較複雜的程序，需要有許多先備知識，未來進階的課程內容有機會會提到。這節將使用 http://gist.github.com (簡稱 gist )來分享你的函式。gist 是一個貼程式碼的網站(像剪貼簿一樣)，你可以直接把上面的程式碼複製貼上(請記得先登入，這樣以後就可以管理你的程式碼)。如下圖：

![](https://github.com/mutolisp/ecoinformatics_course/blob/master/images/paste_to_gist.png?raw=true)

接下來你會在瀏覽器的網址列看到這段函式的網址列(即：https://gist.github.com/mutolisp/643dc4795d0b31f1ef3fdaca1641fa7e)。請先複製起來等會會用到。回到 RStudio 中，請確認一下已經安裝了 [devtools](https://cran.r-project.org/web/packages/devtools/index.html) 套件。

載入 devtools 之後，就可以使用 ```source_gist()``` 來載入剛剛你貼到 gist 的 R function。第一個參數是看起來像亂碼的 hash code，以本例來說就是 643dc4795d0b31f1ef3fdaca1641fa7e，後面的 filename 則是你自行命名的 filename，在這裡則是 ShannonWiener.r

![](https://github.com/mutolisp/ecoinformatics_course/blob/master/images/paste_to_gist2.png?raw=true)

In [8]:
library(devtools)
source_gist('643dc4795d0b31f1ef3fdaca1641fa7e', filename = 'ShannonWiener.r')

ShannonWiener('https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/data/twforest.csv')

Sourcing https://gist.githubusercontent.com/mutolisp/643dc4795d0b31f1ef3fdaca1641fa7e/raw/361b54c7efad1a78d3fd07a7efb1bcab127bc979/ShannonWiener.r
SHA-1 hash of file is 4c62f623ee3f1956b45e37672ed02742b6a18f8a


[1] 2.827801

[本範例 code 下載](https://raw.githubusercontent.com/mutolisp/ecoinformatics_course/master/demos/source_GistFunction.R)

當使用 ```source_gist()``` 後你就直接可以用 ```ShannonWiener``` 函式了！

## 作業

1. 請寫出一個可計算 Shannon-Wiener 多樣性指數和Simpson 多樣性指數(包含 Simpson index, Simpson diversity index, inverse Simpson)的函式，函式必須加入可讓使用者選擇多樣性指數的參數
2. 請寫出一個可計算費布納西(fibonacci)數列的函式